In [ ]:
#>>> I = numpy.asarray(PIL.Image.open('test.jpg'))

#>>> im = PIL.Image.fromarray(numpy.uint8(I))

In [47]:
from PIL import Image 
import numpy
import torch

xs = []
ys = []

for i in range(0, 10):
    img = Image.open(f'images/images/image_{i}.jpeg')
    I = numpy.asarray(img)
    #print("data", I)
    
    xs.append([257,257,257,0,0])
    ys.append(I[0][0].tolist())


    for rows in range(0, 16):
        for cols in range(0,16):
            if rows==15 and cols ==15:
                continue

            temp_x = I[rows][cols].tolist()
            temp_x.append(rows)
            temp_x.append(cols)

            xs.append(temp_x)
            if cols == 15:
                ys.append(I[rows+1][0].tolist())
            else:
                ys.append(I[rows][cols+1].tolist())
            
xs = torch.tensor(xs).float()
ys = torch.tensor(ys).float()

#print("Xs", xs)
#print("Ys", ys)


In [49]:

W1 = torch.randn((5, 300), requires_grad=True)
B1 = torch.randn((300), requires_grad=True)

W2 = torch.randn((300, 3), requires_grad=True)
B2 = torch.randn((3), requires_grad=True)

params = (W1, B1, W2, B2)


for _ in range(5000):
   
    layer1 = xs @ W1 + B1
    tan_layer1 = torch.tanh(layer1)
    
    layer2 = tan_layer1 @ W2 + B2
    
    counts = layer2.exp()
    probs = counts / counts.sum(1, keepdims=True)
    print("probs", probs.size())
    

    sum = 0
    for i in range(probs.shape[0]):
        actual_output = ys[i]
        predicted_prob_of_actual_output = probs[i][actual_output]
        sum += predicted_prob_of_actual_output
    
    sum = sum / probs.shape[0]
    
    neg_log_likelihood = -sum.log()
    
    print("Train #", _ , " Loss: ", neg_log_likelihood.item())

    for param in params:
        param.grad = None
        
    neg_log_likelihood.backward()
    
    for param in params:
        param.data += -.1 * param.grad

probs torch.Size([2560, 3])


IndexError: tensors used as indices must be long, int, byte or bool tensors